<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/fakeclinicalnotes/src/cogs402projecttestnotes/AttributionLongformerAggregateNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook performs [Integrated Gradients](https://arxiv.org/abs/1703.01365) over the entire dataset and aggregates all of the attributions with respect to the positive class. We aggregate using either the complete longformer embeddings, or the word and position embeddings. The notebook outputs a csv file containg tokens and the sum of the attributions over the entire dataset and is used in the [longformer embedding](https://colab.research.google.com/drive/15Zquqi72N2NNusEUXRN53bCKE7qj8KAh?usp=sharing) and [word+positon+token_type embeddings](https://colab.research.google.com/drive/1pptTYAJGp7tl0BhVQoTD5RGyQMEVF766) notebooks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import dependencies

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [ ]:
pip install transformers --quiet

     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 41.1 MB/s 
     |████████████████████████████████| 596 kB 50.2 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 


In [ ]:
pip install captum --quiet

     |████████████████████████████████| 1.4 MB 5.1 MB/s 


In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 141 kB 53.7 MB/s 
     |████████████████████████████████| 115 kB 49.6 MB/s 
     |████████████████████████████████| 212 kB 40.3 MB/s 
     |████████████████████████████████| 127 kB 39.0 MB/s 


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

## Import model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
# model_path = 'danielhou13/longformer-finetuned-news-cogs402'

# load model
test = torch.load("/content/drive/MyDrive/cogs402longformer/fakeclinicalnotes/models/full_augmented_lr2e-5_dropout3_10_trained_threshold.pt")
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', state_dict=test['state_dict'], num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['longformer_model.encoder.layer.3.attention.output.dense.weight', 'longformer_model.encoder.layer.5.output.LayerNorm.weight', 'longformer_model.encoder.layer.4.attention.output.dense.weight', 'longformer_model.encoder.layer.8.attention.self.value.weight', 'longformer_model.encoder.layer.0.intermediate.dense.weight', 'longformer_model.encoder.layer.11.attention.self.key.bias', 'longformer_model.encoder.layer.3.attention.self.query_global.bias', 'longformer_model.encoder.layer.6.attention.self.key_global.bias', 'longformer_model.encoder.layer.4.intermediate.dense.weight', 'longformer_model.encoder.layer.6.output.LayerNorm.bias', 'longformer_model.encoder.layer.11.attention.self.value.weight', 'longformer_model.encoder.layer.8.attention.output.dense.bias', 'longformer_model.encoder.layer.10.attention.self.value.bias', 'longformer_model.encoder.layer.9.

## Import Dataset

Here we import the papers dataset

In [ ]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402datafake")["train"]

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402datafake-c20c2db2d92a66bc


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402datafake-c20c2db2d92a66bc/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Here we import the news dataset

In [ ]:
# cogs402_ds = load_dataset("danielhou13/cogs402dataset2")["validation"]

## Getting the Attributions

For our Integrated Gradients, we need to create a custom forward pass of our model. Specifically we want the softmaxed logits which represent the probability of predicting that class.

In [ ]:
def predict(inputs, position_ids=None, token_type_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   token_type_ids=token_type_ids,
                   attention_mask=attention_mask)
    return output.logits

In [ ]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, token_type_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   token_type_ids=token_type_ids,
                   attention_mask=attention_mask)
    return torch.softmax(preds, dim = 1)

Create functions that give us the input ids, position ids and token_type_ids for the text we want to examine. It also creates a baseline for use in our integrated gradients.

**Note: The function used to create the token type ids is the exact same as the longformer implementation when no token type ids. It is not necessary to create token_type_ids unless you are doing Integrated Gradients using multi-embedding as we need the baselines.**

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

def construct_input_ref_token_type_pair(input_ids):
    seq_len = input_ids.size(1)

    # same as the tensor the model creates when you do not pass in token_type_ids as input.
    token_type_ids = torch.zeros(seq_len, dtype=torch.long, device=device).unsqueeze(0).expand_as(input_ids)
    
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)

    return token_type_ids, ref_token_type_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

Perform Layer Integrated Gradients using the longformer's embeddings. This can easily be adjusted to use longformer word embeddings, position and token_type embeddings.

In [ ]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)
lig2 = LayerIntegratedGradients(custom_forward, \
                                [model.longformer.embeddings.word_embeddings, \
                                 model.longformer.embeddings.position_embeddings,\
                                 model.longformer.embeddings.token_type_embeddings])

/usr/local/lib/python3.7/dist-packages/captum/attr/_core/layer/layer_integrated_gradients.py:103: UserWarning: Multiple layers provided. Please ensure that each layer is**not** solely solely dependent on the outputs ofanother layer. Please refer to the documentation for moredetail.
  "Multiple layers provided. Please ensure that each layer is"


Helper function to sum the attributions and normalize into an array of length (seq_len).

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

We iterate over the entire dataset, getting the input_ids, position_ids and their baselines, performing integrated gradients, summing the attributions, and finally creating a dataframe to store the attributions and respective tokens. After we create the dataframe, get the aggregate attributions for each token in the example and save it in a list of dataframes.

In [ ]:
from tqdm import tqdm
aggregate_attrib = []
aggregation_function = {'attribution': 'sum'}

for i in tqdm(range(len(cogs402_ds))):

  #get input ids, position ids and attention mask for integrated gradients
  text = cogs402_ds[i]['text']
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)

  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)
  all_tokens= [x.replace('Ġ', '') for x in all_tokens]

  # perform integrated gradients
  attributions = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    additional_forward_args=(position_ids, None, attention_mask),
                                    target=1,
                                    n_steps=50,
                                    internal_batch_size = 2)
  
  #get the attributions
  attributions_sum = summarize_attributions(attributions)
  
  #convert into dataframe
  d = {"tokens":all_tokens, "attribution":attributions_sum[:len(all_tokens)].cpu()}  
  df_attrib = pd.DataFrame(d)

  #aggregate the duplicate tokens
  df_attrib = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_function)

  #add to list of dataframes
  aggregate_attrib.append(df_attrib)

100%|██████████| 12/12 [05:25<00:00, 27.15s/it]


Here we have the implementation for the multi-embedding version. The only difference is that we have two attributions that we want to find the aggregate for, the position and word embeddings. 

**Note: despite passing in the token_type_ids and the baseline as inputs, we will not be able to get attributions for it as the input and the baseline are the same. It returns a tensor of nan values.**

We create dataframes for both the word and position attributions to store the attributions and their respective token. We then aggregate the attributions based on the token for both dataframes. Finally, we append the position and word dataframes in their own separate list of dataframes.

In [ ]:
from tqdm import tqdm
# aggregate_attrib = []
# aggregate_pos = []

# aggregation_function = {'attribution': 'sum'}

# for i in tqdm(range(len(cogs402_ds)), position = 0, leave = True):
  
#   #get input_ids, position_ids, and the attention masks for the integrated gradients
#   text = cogs402_ds[i]['text']

#   input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
#   token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids)
#   position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
#   attention_mask = construct_attention_mask(input_ids)

#   indices = input_ids[0].detach().tolist()
#   all_tokens = tokenizer.convert_ids_to_tokens(indices)

#   # compute integrated gradients
#   attributions2 = lig2.attribute(inputs=(input_ids, position_ids, token_type_ids),
#                                baselines=(ref_input_ids, ref_position_ids, ref_token_type_ids),
#                                target=1,
#                                additional_forward_args=attention_mask,
#                                n_steps=20,
#                                internal_batch_size = 2)
  
#   # get the attributions for the words and position ids
#   attributions_word = summarize_attributions(attributions2[0])
#   attributions_position = summarize_attributions(attributions2[1])

#   # convert them both into dataframes 
#   d = {"tokens":all_tokens, "attribution":attributions_word[:len(all_tokens)].cpu()}  
#   d2 = {"tokens":all_tokens, "attribution":attributions_position[:len(all_tokens)].cpu()}  
  
#   df_attrib = pd.DataFrame(d)
#   df_attrib2 = pd.DataFrame(d2)

#   #aggregate the attributions for duplicate tokens
#   df_attrib = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_function)
#   df_attrib2 = df_attrib2.groupby(df_attrib2['tokens']).aggregate(aggregation_function)

#   aggregate_attrib.append(df_attrib)
#   aggregate_pos.append(df_attrib2)

To get the aggregate attributions for every token over the entire dataset, we concatenate the list of dataframes we stored, sum up the attributions of duplicate tokens and divide by the number of items in each list.

In [ ]:
def combinedataframe(listframes, aggregation_func):
  df_attrib = pd.concat(listframes)
  df_attrib = df_attrib.reset_index(level=0)
  df_attrib = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_func)
  df_attrib['attribution'] = df_attrib['attribution'].div(len(listframes))
  highest_attrib_tokens_all = df_attrib.sort_values(by=['attribution'], ascending=False).reset_index()
  return highest_attrib_tokens_all

In [ ]:
df_attrib = combinedataframe(aggregate_attrib, aggregation_function)
# df_attrib_pos = combinedataframe(aggregate_pos, aggregation_function)

## Masking the Tokens

In [ ]:
import nltk
from transformers import AutoTokenizer
nltk.download('stopwords')
tokenizer2 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')
all_stopwords.append(" ")
stopwords = set(tokenizer2.tokenize(all_stopwords, is_split_into_words =True))
stopwords.update(all_stopwords)
print(stopwords)

{'not', 'Ġours', "weren't", 'but', 'Ġno', 'Ġdo', 'ours', 'Ġo', 'Ġcouldn', 'Ġaren', 'Ġdoing', 'having', 'too', 'during', 'Ġherself', 'some', 'Ġtheir', 'Ġabout', 'she', 'Ġthrough', 'any', 'an', 'Ġby', 'Ġown', 'me', 'those', 'Ġdown', 'it', 'both', 'Ġits', 'Ġwho', 'Ġhas', 'up', 'Ġfew', 'Ġthemselves', 'Ġafter', 'Ġs', 'Ġnot', 'Ġmust', 'mustn', 'between', 'where', 'Ġhere', "haven't", 'or', 'most', 'Ġisn', 'Ġsame', 'by', 'didn', 'Ġam', "'ll", 'Ġwe', 'Ġ', "you're", 'they', 'Ġagain', 'mightn', "'d", 'Ġup', 'him', 'Ġtheirs', 'o', 'Ġyour', 'don', 'Ġfrom', 'Ġthese', 'Ġnor', 'Ġhadn', 'now', 'Ġhad', 'wasn', 'the', 'Ġonce', "doesn't", 'again', "she's", 'Ġyourselves', 'he', 'Ġy', 'is', 'hers', 'against', "it's", 'Ġour', 'ourselves', 'Ġoff', "don't", ' ', 'Ġthem', 'Ġhave', 'more', 'these', 'Ġover', 'Ġhe', 'Ġdoes', 'Ġbefore', 'Ġthey', 'i', 'Ġwere', 'Ġeach', "couldn't", 'Ġmy', 're', 'herself', 'been', 'below', 'Ġre', 'shan', 'its', "hadn't", 'under', 'few', "'s", 'Ġsome', 'above', 'Ġit', 'Ġain', 'Ġshe', '

Here we are only showing the top 15 highest attributions, in other words, the tokens that have the most influence in the model predicting positive. If you are running integrated gradients using the longformer embeddings, this will be attributions for those embeddings. If you are running Integrated Gradients using word, position, and token_type embeddings, these will be the word embeddings.

In [ ]:
df_attrib[:20]

,tokens,attribution
0,a,0.718337
1,),0.599780
2,and,0.465760
3,of,0.417988
4,for,0.234652
5,Last,0.203878
6,she,0.169394
7,p,0.167826
8,been,0.166716
9,:,0.165434


In [ ]:
df_attrib[(df_attrib['tokens'].str.isalpha()) & ~(df_attrib['tokens'].isin(stopwords))][:20].reset_index(drop=True)

,tokens,attribution
0,Last,0.203878
1,p,0.167826
2,id,0.105127
3,parents,0.100191
4,ox,0.096997
5,Pattern,0.092647
6,He,0.074614
7,family,0.069991
8,mg,0.068563
9,ine,0.066980


Here we are showing the 15 highest attributions for the position embeddings. Note that running integrated gradients using the longformer embeddings rather than the word, position and token_type embeddings will not have this output.

In [ ]:
# df_attrib_pos[:15]

Here we are only showing the top 15 lowest attributions, in other words, the tokens that have the most influence in the model predicting negative. If you are running integrated gradients using the longformer embeddings, this will be attributions for those embeddings. If you are running Integrated Gradients using word, position and token_type embeddings, these will be the word embeddings.

In [ ]:
df_attrib[:-19:-1]

,tokens,attribution
2861,**,-2.215406
2860,,-1.319923
2859,.,-1.213790
2858,(,-0.817659
2857,in,-0.748575
2856,",",-0.608259
2855,],-0.567021
2854,-,-0.489356
2853,Name,-0.455484
2852,the,-0.409331


In [ ]:
df_attrib[(df_attrib['tokens'].str.isalpha()) & ~(df_attrib['tokens'].isin(stopwords))][:-19:-1].reset_index(drop=True)

,tokens,attribution
0,Name,-0.455484
1,CHAR,-0.190179
2,well,-0.149171
3,DIS,-0.147607
4,per,-0.106924
5,itle,-0.102404
6,discharge,-0.100078
7,un,-0.096693
8,daily,-0.086808
9,time,-0.086199


Here we are showing the 15 lowest attributions for the position embeddings. Note that running integrated gradients using the longformer embeddings rather than the word, position and token_type embeddings will not have this output.

In [ ]:
# df_attrib_pos[:-14:-1]

Save the pandas dataframe into a csv to access it in the future without having to run through the entire dataset. Change the path and file name to one fitting your project.

In [ ]:
# # longformer embeddings
# df_attrib.to_csv('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/longformer_emb_papers.csv')
# df_attrib.to_csv('/content/drive/MyDrive/cogs402longformer/results/news/news_attributions/longformer_emb_news.csv')

# Word + position embeddings for the papers dataset
# df_attrib.to_csv('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/word_emb_papers.csv')
# df_attrib_pos.to_csv('/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/pos_emb_papers.csv')

# # # Word + position embeddings for the news dataset
# df_attrib.to_csv('/content/drive/MyDrive/cogs402longformer/results/news/news_attributions/word_emb_news.csv')
# df_attrib_pos.to_csv('/content/drive/MyDrive/cogs402longformer/results/news/news_attributions/pos_emb_news.csv')